Credit: https://www.kaggle.com/abhishek/step-1-create-folds

In [ ]:
import os
import random
import gc

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
sns.set(style='darkgrid')

from sklearn.model_selection import StratifiedKFold

In [ ]:
TRAIN = '../input/commonlitreadabilityprize/train.csv'
SEED = 28

df = pd.read_csv(TRAIN)

In [ ]:
def get_double_stratified(df, n_splits=5):
    n_bins_outer = int(np.floor(1 + np.log2(len(df))))
    df['outer_bin'] = pd.cut(df.target, n_bins_outer, labels=False)
    df['inner_bin'] = np.nan
    df['fold'] = np.nan

    skf = StratifiedKFold(n_splits=n_splits, random_state=SEED, shuffle=True)    
    skf_df = pd.DataFrame()
    
    for outer_bin in range(n_bins_outer):
        temp_df = df.loc[df.outer_bin==outer_bin].reset_index(drop=True)
        n_bins_inner = int(np.floor(1 + np.log2(len(temp_df))))
        
        temp_df['inner_bin'] = pd.cut(temp_df.standard_error, n_bins_inner, labels=False)
        gen_skf = skf.split(temp_df.id, y=temp_df.inner_bin)
        
        for fold, (idx_tr, idx_val) in enumerate(gen_skf):
            temp_df.loc[idx_val, 'fold'] = fold
            skf_df = pd.concat([skf_df, temp_df.loc[idx_val]], axis=0)
        
    skf_df = skf_df.reset_index(drop=True)
    skf_df['fold'] = skf_df['fold'].astype('int16')
    return skf_df

In [ ]:
df = get_double_stratified(df, n_splits=5)
df.head()